In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   445   0  0   1060   1061  1059  0.01  0.36  0.45  0.79  0.78
1   475   1  0   1014   1018  1017  0.01  0.38  0.45  0.81  0.81
2   481   2  0    983    989   987  0.02  0.39  0.45  0.80  0.80
3   484   3  0    959    967   966  0.02  0.41  0.45  0.78  0.77
4   487   4  0    938    948   947  0.02  0.43  0.45  0.75  0.74
5   505   5  0    923    934   933  0.03  0.44  0.45  0.78  0.78
6   503   6  0    914    923   923  0.03  0.44  0.45  0.77  0.77
7   503   7  0    907    916   916  0.03  0.44  0.45  0.77  0.77
8   510   8  0    903    912   912  0.03  0.43  0.45  0.77  0.77
9   506   9  0    901    910   910  0.03  0.43  0.45  0.77  0.77
10  523  10  0    899    908   908  0.04  0.43  0.45  0.76  0.76
11  444  11  0    897    906   905  0.04  0.42  0.45  0.76  0.75
12  521  12  0    893    902   902  0.04  0.42  0.45  0.75  0.74
13  439  13  0    892    902   901  0.04  0.42  0.45  0.74  0.74
14  431  14  0    891    901   901  0.04  0.42  0.45  0.74  0.74
15  426  15  0    891    900   900  0.04  0.41  0.45  0.74  0.74
16  428  16  0    888    898   898  0.04  0.41  0.45  0.74  0.73
17  428  17  0    882    891   891  0.04  0.40  0.45  0.72  0.72
18  422  18  0    868    877   877  0.04  0.39  0.45  0.71  0.71
19  423  19  0    862    870   870  0.05  0.38  0.45  0.71  0.70
20  422  20  0    859    868   867  0.05  0.38  0.45  0.70  0.70
21  425  21  0    859    867   866  0.05  0.38  0.45  0.70  0.70
22  430  22  0    857    865   865  0.05  0.38  0.45  0.70  0.70
23  429  23  0    857    865   864  0.05  0.38  0.45  0.70  0.70
24  429  24  0    856    864   864  0.05  0.38  0.45  0.70  0.70
25  428  25  0    856    864   863  0.05  0.38  0.45  0.70  0.70
26  431  26  0    856    863   863  0.06  0.38  0.45  0.70  0.70
27  427  27  0    856    863   863  0.06  0.38  0.45  0.70  0.70
28  362  28  0    855    863   863  0.06  0.38  0.45  0.70  0.70
29  438  29  0    855    862   862  0.06  0.38  0.45  0.70  0.70
30  356  30  0    855    863   862  0.06  0.38  0.45  0.70  0.70
31  359  31  0    855    863   862  0.06  0.38  0.45  0.70  0.70
32  353  32  0    855    863   862  0.06  0.38  0.45  0.70  0.70
33  357  33  0    856    863   862  0.06  0.38  0.45  0.70  0.70
34  353  34  0    856    864   862  0.06  0.38  0.45  0.70  0.70
35  358  35  0    856    864   862  0.06  0.38  0.45  0.70  0.70
36  357  36  0    856    864   862  0.06  0.38  0.45  0.70  0.70
37  361  37  0    857    864   862  0.06  0.38  0.45  0.70  0.70
38  355  38  0    857    865   862  0.06  0.38  0.45  0.70  0.70
39  356  39  0    857    864   862  0.06  0.38  0.45  0.70  0.70
40  445  40  0    853    860   860  0.06  0.38  0.45  0.69  0.69
41  439  41  0    851    859   859  0.06  0.38  0.45  0.69  0.69
42  440  42  0    850    858   859  0.06  0.38  0.45  0.69  0.69
43  441  43  0    850    858   858  0.06  0.38  0.45  0.69  0.68
44  372  44  0    850    858   858  0.06  0.38  0.45  0.69  0.69
45  450  45  0    849    858   858  0.06  0.38  0.45  0.69  0.69
46  362  46  0    850    858   858  0.06  0.38  0.45  0.69  0.69
47  437  47  0    849    858   857  0.06  0.38  0.45  0.69  0.69
48  461  48  0    849    858   857  0.06  0.38  0.45  0.69  0.68
49  376  49  0    849    858   857  0.06  0.38  0.45  0.69  0.68

 0 11258 349692.0 write get article one use like know go think make
   0-1 4585 335022.0 say people one god write think know make go article
     0-1-1 4507 3911.0 sky nsa kent satellite secure liberty flight theory british criminal
     0-1-2 25 21.0 door god hall summary knife louis illegal work home nature
     0-1-3 53 16.0 myers america secretary apr news change boston finish want order
   0-2 4100 162942.0 use file program key window system available information image windows
     0-2-1 4051 1706.0 steal criminal mb warrant technology drug threat work focus professional
     0-2-2 49 17.0 movie hardly mount college absolutely cut stanley pull message ok
   0-3 1897 72812.0 game team play year player win go get season hockey
     0-3-1 1870 1323.0 excellent doug module miss oo dod fair cover poster fall
     0-3-2 27 50.0 sin double int printf speed moment kind require address else
   0-4 16 2378.0 db mov bh cs rg byte oo al de eus
     0-4-1 16 1.0 de la administration bob ram da

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta20-gam0.01-epoch50/log' mode='wb' closefd=True>
